In [ ]:
"""
Лабораторная №1
Выполнили студенты группы М8О-102СВ-25
Тысячный Владислав
Гуреева Алина
"""


import vk_api
import time
import pandas as pd

vk_key = "vk1.a.xjRAlVzLY1LM0aNBnE1AWmPeQlzDnck6zC5yrFgjxYFf38Wg3Ecl9BrvYuBDIRDnTz15KawnW9cEx7d1hFOCIKpmM45qUlk7-SFEqcVcOrVMHbVJJ2ScE_rpJrU3ZPTq9NxqMDepKr4pubNjIMZkDc1EECyImoYXFFypmYOZZkZbm_jcL4F1yzAfz8Ui5MaD5Z4Qiuko1gTmpTCWWYqJsg"
targetUser = input("Введите ID или ник пользователя: ")
maxDepth = 3 # глубина (1 — друзья, 2 — друзья друзей) 
sleep = 0.3 

vk = vk_api.VkApi(token=vk_key)
api = vk.get_api()

all_users = set()
contacts = []

def get_user_id(user):
    if user.isdigit():
        return int(user)
    else:
        info = api.users.get(user_ids=user)
        return info[0]['id']
    

def get_friends(user_id):
    try:
        friends = api.friends.get(user_id=user_id)['items']
        return friends
    except:
        return []
    

def crawl(user_id, depth):
    # Если глубина больше максимальной, выходим
    if depth > maxDepth:
        return

    # Если пользователя уже добавили, тоже выходим
    if user_id in all_users:
        return

    # Добавляем пользователя в список всех пользователей
    all_users.add(user_id)

    # Получаем список друзей
    friends = get_friends(user_id)

    # Сохраняем связи 
    for f in friends:
        contacts.append((user_id, f))

 
    print("Пользователь", user_id, "друзей найдено:", len(friends))


    time.sleep(sleep)

    # Рекурсивно обходим друзей
    for f in friends:
        crawl(f, depth + 1)

user_id = get_user_id(targetUser)
if user_id:
    crawl(user_id, 1)
    print(f"\nВсего собрано {len(all_users)} пользователей и {len(contacts)} связей")

    # матрица смежности для модели
    users_list = list(all_users)
    n = len(users_list)
    index = {u: i for i, u in enumerate(users_list)}
    matrix = [[0]*n for _ in range(n)]

    for j, k in contacts:
        if j in index and k in index:
            i, j = index[j], index[k]
            matrix[i][j] = 1
            matrix[j][i] = 1

    df = pd.DataFrame(matrix, index=users_list, columns=users_list)
    df.to_csv("adjacency_matrix.csv", index=True)
    print(f"\nМатрица смежности сохранена в adjacency_matrix.csv")
    
    df.to_excel("adjacency_matrix.xlsx", index=True)
    print(f"\nМатрица смежности сохранена в adjacency_matrix.xlsx (Excel)")
else:
    print("Неверный ID или ник.")


Пользователь 273359617 друзей найдено: 103
Пользователь 1999634 друзей найдено: 377
Пользователь 132540 друзей найдено: 0
Пользователь 149409 друзей найдено: 0
Пользователь 191954 друзей найдено: 288
Пользователь 195320 друзей найдено: 0
Пользователь 204782 друзей найдено: 290
Пользователь 219916 друзей найдено: 601
Пользователь 240834 друзей найдено: 96
Пользователь 250851 друзей найдено: 412
Пользователь 264286 друзей найдено: 160
Пользователь 300382 друзей найдено: 540
Пользователь 445068 друзей найдено: 538
Пользователь 602438 друзей найдено: 257
Пользователь 661448 друзей найдено: 0
Пользователь 674174 друзей найдено: 175
Пользователь 851860 друзей найдено: 197
Пользователь 877026 друзей найдено: 0
Пользователь 939951 друзей найдено: 325
Пользователь 973266 друзей найдено: 0
Пользователь 996815 друзей найдено: 148
Пользователь 1004315 друзей найдено: 500
Пользователь 1017642 друзей найдено: 0
Пользователь 1040198 друзей найдено: 0
Пользователь 1056995 друзей найдено: 206
Пользоват

In [1]:
import numpy as np
import pandas as pd
import networkx as nx

df = pd.read_csv("adjacency_matrix.csv")
adj_matrix = np.array(df.iloc[:, 1:])
print(adj_matrix.shape)
G = nx.from_numpy_array(adj_matrix)

def closeness_centrality(G):
    """
    Вычисляет центральность по близости
    """
    closeness = nx.closeness_centrality(G)
    
    n = G.number_of_nodes()
    closeness_array = np.array([closeness[i] for i in range(n)])
    
    return closeness_array

def betweenness_centrality(G, normalized=True):
    """
    Вычисляет центральность по посредничеству
    """
    betweenness = nx.betweenness_centrality(G, normalized=normalized)
    
    n = G.number_of_nodes()
    betweenness_array = np.array([betweenness[i] for i in range(n)])
    
    return betweenness_array

def eigenvector_centrality(G, max_iter=1000):
    """
    Вычисляет центральность по собственным векторам
    """
    n = G.number_of_nodes()
    try:
        eigenvector = nx.eigenvector_centrality(G, max_iter=max_iter)

        eigenvector_array = np.array([eigenvector[i] for i in range(n)])
    except:
        # Если вдруг алгоритм не сходится, возвращаем нулевой вектор
        eigenvector_array = np.zeros(n)
    
    return eigenvector_array


(9064, 9064)


In [2]:
import numpy as np
import pandas as pd
import random
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score, f1_score
import networkx as nx

def extract_features(adj_matrix):
    """
    Извлекает признаки для всех возможных пар узлов
    """
    G = nx.from_numpy_array(adj_matrix)
    n = G.number_of_nodes()
    
    features = []
    labels = []

    common_friends = adj_matrix @ adj_matrix
    common_friends_2 = common_friends @ adj_matrix
    eigenv_centrality = eigenvector_centrality(G)
    
    for i in range(n):
        for j in range(i + 1, n):
            feature_vector = []
            
            # Общие друзья 1 порядка
            feature_vector.append(common_friends[i][j])

            # Общие друзья 2 порядка
            feature_vector.append(common_friends_2[i][j])
            
            # Коэффициент Жаккара
            friends_i = set(G.neighbors(i))
            friends_j = set(G.neighbors(j))
            if len(friends_i | friends_j) > 0:
                jaccard = len(friends_i & friends_j) / len(friends_i | friends_j)
            else:
                jaccard = 0
            feature_vector.append(jaccard)
            
            # Кратчайшее расстояние
            try:
                shortest_path = nx.shortest_path_length(G, i, j)
            except:
                shortest_path = n  # максимальное возможное расстояние
            feature_vector.append(shortest_path)
            
            # Разность центральностей по собственным векторам
            feature_vector.append(abs(eigenv_centrality[i] - eigenv_centrality[j]))
            
            features.append(feature_vector)
            labels.append(adj_matrix[i, j])
    
    return np.array(features), np.array(labels)

def create_test(adj_matrix):
    """
    Создает из матрицы смежности тестовые данные для обучения
    """

    n = adj_matrix.shape[0]

    y_test = []
    friends_pair = []
    
    # Получение y_test
    for i in range(n):
        for j in range(i + 1, n):
            if adj_matrix[i][j] == 1:
                friends_pair.append((i, j))
            y_test.append(adj_matrix[i][j])

    # Отчистка k дружеских связей
    k = 0.3
    del_pair = random.sample(friends_pair, int(len(friends_pair) * k))
    for i, j in del_pair:
        adj_matrix[i][j] = 0

    # Получение X_test
    X_test, _ = extract_features(adj_matrix)

    return X_test, y_test
    


In [3]:
def train_models(X_train, y_train):
    """
    Обучает несколько моделей для сравнения
    """
    models = {
        'Logistic Regression': LogisticRegression(random_state=42, max_iter=30),
        'Random Forest': RandomForestClassifier(n_estimators=2, random_state=42),
        'Gradient Boosting': GradientBoostingClassifier(n_estimators=5, random_state=42),
        'Neural Network': MLPClassifier(hidden_layer_sizes=(16, 8), random_state=42, max_iter=10)
    }
    
    trained_models = {}
    for name, model in models.items():
        model.fit(X_train, y_train)
        trained_models[name] = model
    
    return trained_models

def evaluate_models(models, X_test, y_test):
    """
    Оценивает качество моделей
    """
    results = {}
    for name, model in models.items():
        y_pred_proba = model.predict_proba(X_test)[:, 1]
        y_pred = model.predict(X_test)
        
        results[name] = {
            'ROC-AUC': roc_auc_score(y_test, y_pred_proba),
            'F1-Score': f1_score(y_test, y_pred)
        }
    
    return pd.DataFrame(results).T



In [4]:
def friendship_prediction_pipeline(adj_matrix_train, adj_matrix_test):
    """
    Полный пайплайн предсказания дружбы
    """
    # Извлекаем признаки
    print("Извлечение признаков из обучающей выборки...")
    X_train, y_train = extract_features(adj_matrix_train)
    
    # Обучение моделей
    print("Обучение моделей...")
    models = train_models(X_train, y_train)

    # Подготовка тестовой выборки
    print("Подготовка тестовой выборки...")
    X_test, y_test = create_test(adj_matrix_test)
    
    
    # Оценка
    print("Оценка моделей...")
    results = evaluate_models(models, X_test, y_test)
    
    return results

# Пример использования
if __name__ == "__main__":

    df_train = pd.read_csv("adjacency_matrix_train.csv")
    adj_matrix_train = np.array(df_train.iloc[:, 1:])

    df_test1 = pd.read_csv("adjacency_matrix_test1.csv")
    adj_matrix_test1 = np.array(df_test1.iloc[:, 1:])

    df_test2 = pd.read_csv("adjacency_matrix_test2.csv")
    adj_matrix_test2 = np.array(df_test2.iloc[:, 1:])

    df_test3 = pd.read_csv("adjacency_matrix_test3.csv")
    adj_matrix_test3 = np.array(df_test3.iloc[:, 1:])

    df_test = pd.read_csv("adjacency_matrix.csv")
    adj_matrix_test = np.array(df_test.iloc[:, 1:])

    
    results = friendship_prediction_pipeline(adj_matrix_train, adj_matrix_test1)
    print()
    
    print("\nРезультаты:")
    print(results)

    results = friendship_prediction_pipeline(adj_matrix_train, adj_matrix_test2)
    print()
    
    print("\nРезультаты:")
    print(results)
    
    results = friendship_prediction_pipeline(adj_matrix_train, adj_matrix_test3)
    print()

    print("\nРезультаты:")
    print(results)

    results = friendship_prediction_pipeline(adj_matrix_train, adj_matrix_test)
    print()
    
    print("\nРезультаты:")
    print(results)

Извлечение признаков из обучающей выборки...
Обучение моделей...


C:\Users\Tysyachnyj V\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Tysyachnyj V\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


Подготовка тестовой выборки...
Оценка моделей...


Результаты:
                      ROC-AUC  F1-Score
Logistic Regression  0.999890  0.103053
Random Forest        0.999994  0.964583
Gradient Boosting    1.000000  1.000000
Neural Network       0.999996  0.935760
Извлечение признаков из обучающей выборки...
Обучение моделей...


C:\Users\Tysyachnyj V\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Tysyachnyj V\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


Подготовка тестовой выборки...
Оценка моделей...


Результаты:
                      ROC-AUC  F1-Score
Logistic Regression  0.998804  0.299582
Random Forest        0.999993  0.981454
Gradient Boosting    1.000000  1.000000
Neural Network       0.999996  0.971660
Извлечение признаков из обучающей выборки...
Обучение моделей...


C:\Users\Tysyachnyj V\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Tysyachnyj V\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


Подготовка тестовой выборки...
Оценка моделей...


Результаты:
                      ROC-AUC  F1-Score
Logistic Regression  0.996945  0.600495
Random Forest        0.999997  0.991323
Gradient Boosting    1.000000  1.000000
Neural Network       0.999997  0.990049
Извлечение признаков из обучающей выборки...
Обучение моделей...


C:\Users\Tysyachnyj V\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Tysyachnyj V\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


Подготовка тестовой выборки...
Оценка моделей...


Результаты:
                      ROC-AUC  F1-Score
Logistic Regression  0.999054  0.550783
Random Forest        1.000000  0.994284
Gradient Boosting    1.000000  1.000000
Neural Network       0.999809  0.891569
